<a href="https://colab.research.google.com/github/letthai/Movie-Genre-Classification/blob/main/Group_8_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

In [ ]:
import pandas
import numpy as np
import math
import ast
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import matplotlib.pyplot as plt
from operator import itemgetter
import cv2
import os, io
from nltk import wordpunct_tokenize
import re
import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
from torch import optim

!pip install -q torchmetrics
from torchmetrics import Precision, Recall, Accuracy, F1Score
from torchmetrics.functional.retrieval import retrieval_average_precision

import networkx as nx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 5.3 MB/s eta 0:00:00


# Part 1: Data preparation

## Tables construction

In this part, we constructed and used three main table, including: movies_train, movies_test, and ratings (more details in our report)

In [ ]:
!gdown 1hUqu1mbFeTEfBvl-7fc56fHFfCSzIktD

Downloading...
From: https://drive.google.com/uc?id=1hUqu1mbFeTEfBvl-7fc56fHFfCSzIktD
To: /content/ml1m.zip
100% 105M/105M [00:00<00:00, 133MB/s] 


In [ ]:
!unzip -qq ml1m.zip -d ml1m

In [ ]:
# You need extract file ml1m.zip to folder ml1m before run code

users = pandas.read_csv('ml1m/content/dataset/users.dat', sep='::',
                        engine='python',
                        names=['userid', 'gender', 'age', 'occupation', 'zip']).set_index('userid')
ratings = pandas.read_csv('ml1m/content/dataset/ratings.dat', engine='python',
                          sep='::', names=['userid', 'movieid', 'rating', 'timestamp'])
movies_train = pandas.read_csv('ml1m/content/dataset/movies_train.dat', engine='python',
                         sep='::', names=['movieid', 'title', 'genre'], encoding='latin-1', index_col=False).set_index('movieid')
movies_test = pandas.read_csv('ml1m/content/dataset/movies_test.dat', engine='python',
                         sep='::', names=['movieid', 'title', 'genre'], encoding='latin-1', index_col=False).set_index('movieid')
movies_train['genre'] = movies_train.genre.str.split('|')
movies_test['genre'] = movies_test.genre.str.split('|')

users.age = users.age.astype('category')
users.gender = users.gender.astype('category')
users.occupation = users.occupation.astype('category')
ratings.movieid = ratings.movieid.astype('category')
ratings.userid = ratings.userid.astype('category')


In [ ]:
folder_img_path = 'ml1m/content/dataset/ml1m-images'
movies_train['id'] = movies_train.index
movies_train.reset_index(inplace=True)
movies_train['img_path'] = movies_train.apply(lambda row: os.path.join(folder_img_path, f'{row.id}.jpg'), axis = 1)
movies_train

,movieid,title,genre,id,img_path
0,1650,Washington Square (1997),[Drama],1650,ml1m/content/dataset/ml1m-images/1650.jpg
1,185,"Net, The (1995)","[Sci-Fi, Thriller]",185,ml1m/content/dataset/ml1m-images/185.jpg
2,1377,Batman Returns (1992),"[Action, Adventure, Comedy, Crime]",1377,ml1m/content/dataset/ml1m-images/1377.jpg
3,3204,"Boys from Brazil, The (1978)",[Thriller],3204,ml1m/content/dataset/ml1m-images/3204.jpg
4,1901,Dear Jesse (1997),[Documentary],1901,ml1m/content/dataset/ml1m-images/1901.jpg
...,...,...,...,...,...
3101,2539,Analyze This (1999),[Comedy],2539,ml1m/content/dataset/ml1m-images/2539.jpg
3102,3038,"Face in the Crowd, A (1957)",[Drama],3038,ml1m/content/dataset/ml1m-images/3038.jpg
3103,1832,Heaven's Burning (1997),"[Action, Drama]",1832,ml1m/content/dataset/ml1m-images/1832.jpg
3104,657,Yankee Zulu (1994),"[Comedy, Drama]",657,ml1m/content/dataset/ml1m-images/657.jpg


In [ ]:
folder_img_path = 'ml1m/content/dataset/ml1m-images'
movies_test['id'] = movies_test.index
movies_test.reset_index(inplace=True)
movies_test['img_path'] = movies_test.apply(lambda row: os.path.join(folder_img_path, f'{row.id}.jpg'), axis = 1)
movies_test

,movieid,title,genre,id,img_path
0,3397,"Great Muppet Caper, The (1981)","[Children's, Comedy]",3397,ml1m/content/dataset/ml1m-images/3397.jpg
1,2067,Doctor Zhivago (1965),"[Drama, Romance, War]",2067,ml1m/content/dataset/ml1m-images/2067.jpg
2,2651,Frankenstein Meets the Wolf Man (1943),[Horror],2651,ml1m/content/dataset/ml1m-images/2651.jpg
3,2989,For Your Eyes Only (1981),[Action],2989,ml1m/content/dataset/ml1m-images/2989.jpg
4,3415,"Mirror, The (Zerkalo) (1975)",[Drama],3415,ml1m/content/dataset/ml1m-images/3415.jpg
...,...,...,...,...,...
772,2309,"Inheritors, The (Die Siebtelbauern) (1998)",[Drama],2309,ml1m/content/dataset/ml1m-images/2309.jpg
773,2421,"Karate Kid, Part II, The (1986)","[Action, Adventure, Drama]",2421,ml1m/content/dataset/ml1m-images/2421.jpg
774,3255,"League of Their Own, A (1992)","[Comedy, Drama]",3255,ml1m/content/dataset/ml1m-images/3255.jpg
775,974,Algiers (1938),"[Drama, Romance]",974,ml1m/content/dataset/ml1m-images/974.jpg


In [ ]:
dataset = [movies_train, movies_test]
dataset = pd.concat(dataset)
dataset = dataset[['movieid', 'genre', 'img_path']]

## Dataset preparation

In [ ]:
def tokenize(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    tokens = wordpunct_tokenize(text)
    tokens = tokens[:-1] # remove last token because it is the year which maybe is not useful
    return tokens

def create_vocab():
    df = movies_train.copy()
    arr_title = df['title'].tolist()
    vocab = set()
    for title in arr_title:
        tokens = tokenize(title)
        vocab.update(tokens)
    vocab = list(vocab)
    pad_token = '<PAD>'
    unk_token = '<UNK>'
    vocab.append(pad_token)
    vocab.append(unk_token)
    return vocab

In [ ]:
class MLDataset(Dataset):
    def __init__(self, is_train=True):
        if is_train:
            self.data =  movies_train
        else:
            self.data = movies_test
        self.data['title_tokens'] = [tokenize(x) for x in self.data.title]

        # create vocab
        vocab = create_vocab()
        pad_token = '<PAD>'
        unk_token = '<UNK>'
        self.token2idx = {token: idx for idx, token in enumerate(vocab)}

        # Create a binary vector for each word in each sentence
        MAX_LENGTH = 7
        vectors = []
        for title_tokens in self.data.title_tokens.tolist():
            if len(title_tokens) < MAX_LENGTH:
                num_pad = MAX_LENGTH - len(title_tokens)
                for idx in range(num_pad):
                    title_tokens.append(pad_token)
            else:
                title_tokens = title_tokens[:MAX_LENGTH]
            title_vectors = []
            for word in title_tokens:
                binary_vector = np.zeros(len(vocab))
                if word in vocab:
                    binary_vector[self.token2idx[word]] = 1
                else:
                    binary_vector[self.token2idx[unk_token]] = 1
                title_vectors.append(binary_vector)

            vectors.append(np.array(title_vectors))
        self.data['vectors'] = vectors

        # label genre
        with open('ml1m/content/dataset/genres.txt', 'r') as f:
            genre_all = f.readlines()
            genre_all = [x.replace('\n','') for x in genre_all]
        self.genre2idx = {genre:idx for idx, genre in enumerate(genre_all)}

    def __getitem__(self, index):
        title = self.data.iloc[index].title
        img_path = self.data.iloc[index].img_path
        genre = self.data.iloc[index].genre
        movieid = self.data.iloc[index].movieid

        # preprocess text
        title_vector = self.data.iloc[index].vectors
        title_tensor = torch.from_numpy(title_vector).float()

        # preprocess img
        img_tensor = None

        if os.path.exists(img_path):
            img = cv2.imread(img_path)
            img = img / 255
            img = cv2.resize(img, (256,256))
            img_tensor = torch.from_numpy(img.transpose(2,0,1)).float()
        else:
            img_tensor = torch.zeros((3, 256, 256))

        # preprocess label
        genre_vector = np.zeros(len(self.genre2idx))

        for g in genre:
            genre_vector[self.genre2idx[g]] = 1
        genre_tensor = torch.from_numpy(genre_vector).float()

        return title_tensor, img_tensor, genre_tensor, movieid

    def __len__(self):
        return len(self.data)

In [ ]:
train_set = MLDataset(is_train=True)
test_set = MLDataset(is_train=False)

BATCH_SIZE = 16
train_dataloader = DataLoader(train_set, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_set, batch_size=BATCH_SIZE)

## Graph construction

We constructed a movie-movie similarity network. For each pair of movies, we calculated two ratings-based similarity measures: Pearson correlation coefficient $\rho$, and Jaccard index of common users $\phi$, and a genres-based similarity for pairs of movies $\gamma$ (The equation is defined in our report). The result is a large undirected network whose nodes are movies with labels (train or test movie), and the edges are equipped with three weight: $\rho$, $\phi$, and $\gamma$.

### Graph construction process (You **do not have** to run this part, it takes you lot of time)

Functions calculating pearson, jaccard, and $\gamma$ (genres-related) between two movies.

In [ ]:
value_counts = ratings['movieid'].value_counts()

# Create a new DataFrame from the value_counts
result_df = pandas.DataFrame({'movieid': value_counts.index, 'Count': value_counts.values})

# Sort the result DataFrame by 'MovieID'
result_df = result_df.sort_values(by='movieid')

sum_ratings_by_movie = ratings.groupby('movieid')['rating'].sum().reset_index(name='SumRatings')

result_df = pandas.merge(result_df, sum_ratings_by_movie, on='movieid')
result_df['average'] = result_df['SumRatings'] / result_df['Count']

def count_values(cell):
    return len(cell)

# Apply the function to the column with object type
movies_train['Count_of_Values'] = movies_train['genre'].apply(lambda x: count_values(x))

# Display the DataFrame
print(movies_train)

# Apply the function to the column with object type
movies_test['Count_of_Values'] = movies_test['genre'].apply(lambda x: count_values(x))

In [ ]:
def jaccard_btw_two_movies(movie1, movie2):
  rate_movie_1 = ratings[ratings.movieid == movie1]['userid']
  rate_movie_1 = rate_movie_1.array
  count1 = len(rate_movie_1)

  rate_movie_2 = ratings[ratings.movieid == movie2]['userid']
  rate_movie_2 = rate_movie_2.array
  count2 = len(rate_movie_2)

  rateboth = [x for x in rate_movie_1 if x in rate_movie_2]

  if (count1 + count2 - len(rateboth) == 0) or (len(rateboth) == 0):
    return 0

  return (len(rateboth)/(count1 + count2 - len(rateboth)))

def pearson(movie_m, movie_n):
  # Calculate the average rate of each film
  average_m = result_df[result_df.movieid == movie_m]['average'].values
  average_n = result_df[result_df.movieid == movie_n]['average'].values

  # Get list user rate 2 film
  rateA = ratings[ratings.movieid == movie_m]['userid']
  rateA = rateA.array
  rateB = ratings[ratings.movieid == movie_n]['userid']
  rateB = rateB.array
  rateboth = [x for x in rateA if x in rateB]

  # Calculate the pearson value
  if(len(rateboth) == 0):
    return 0
  else:
    sum1 = 0
    sum2 = 0
    sum3 = 0
    for id in rateboth:
      rating_value_m = ratings[(ratings['userid'] == id) & (ratings['movieid'] == movie_m)]['rating'].values
      rating_value_n = ratings[(ratings['userid'] == id) & (ratings['movieid'] == movie_n)]['rating'].values
      sum1 = sum1 + (rating_value_m - average_m)*(rating_value_n - average_n)
      sum2 = sum2 + (rating_value_m - average_m)*(rating_value_m - average_m)
      sum3 = sum3 + (rating_value_n - average_n)*(rating_value_n - average_n)
    pearson_value = sum1 / (math.sqrt(sum2 * sum3))
    return pearson_value.item() if isinstance(pearson_value, np.ndarray) else pearson_value

def calculate_gamma(movie_m, movie_n):
    is_train_m = movie_m in movies_train.index
    is_train_n = movie_n in movies_train.index

    if is_train_m:
        values_movieid1 = set(movies_train.loc[movies_train.index == movie_m, 'genre'].explode())
        g_m = movies_train[movies_train.index == movie_m]['Count_of_Values'].values
    else:
        values_movieid1 = set(movies_test.loc[movies_test.index == movie_m, 'genre'].explode())
        g_m = movies_test[movies_test.index == movie_m]['Count_of_Values'].values

    if is_train_n:
        values_movieid2 = set(movies_train.loc[movies_train.index == movie_n, 'genre'].explode())
        g_n = movies_train[movies_train.index == movie_n]['Count_of_Values'].values
    else:
        values_movieid2 = set(movies_test.loc[movies_test.index == movie_n, 'genre'].explode())
        g_n = movies_test[movies_test.index == movie_n]['Count_of_Values'].values

    common_values_count = len(values_movieid1.intersection(values_movieid2))

    if common_values_count == 0:
        return 0
    else:
        value = (2 * abs(common_values_count)) / (abs(g_m) + abs(g_n))
        return value.item() if isinstance(value, np.ndarray) else value

Constructing graph

In [ ]:
# G = nx.Graph()
# movie_ids_train = movies_train.index
# movie_ids_test  = movies_test.index
# for i in movie_ids_train:
#   G.add_node(i, value = "train")
# for i in movie_ids_test:
#   G.add_node(i, value = "test")
# for i in range(1, 3953):
#   for j in range(i + 1, 3953):
#     value1 = pearson(i, j)
#     value2 = jaccard_btw_two_movies(i, j)
#     value3 = calculate_gamma(i, j)
#     if value1 > 0:
#       G.add_edge(i, j, weight = [value1, value2, value3])
#       print(i)
#       print(j)
#       print(G.edges[(i, j)])

### Load the graph (Run this part, we have constructed the graph and save it to a pickle file, for saving time)

In [ ]:
!gdown 1iUzPVhvYBVlNhIy9cvBkZIbeYqKgAMuw

Downloading...
From: https://drive.google.com/uc?id=1iUzPVhvYBVlNhIy9cvBkZIbeYqKgAMuw
To: /content/G00013952-final.pkl
100% 179M/179M [00:02<00:00, 74.9MB/s]


In [ ]:
with open('/content/G00013952-final.pkl', 'rb') as f:
  G = pickle.load(f)

print(G.number_of_edges(), G.number_of_nodes())

3956045 3883


# Proposed model

## Predict genres movie based on user-ratings

### Define K-NN for finding all neighborhoods of a movie, and predict genres based on genres of neighborhoods. (You do not have to run this cell)

In [ ]:
def knn(graph, node, n, test_id):
    sorted_edges = sorted([(e[2]['weight'][0] * e[2]['weight'][1], e[1])
                           for e in graph.edges(node, data=True) if e[1] not in test_id and e[2]['weight'][2] > 0])
    selected_nodes = [edge[1] for edge in sorted_edges[:n]]
    return selected_nodes

In [ ]:
def number_of_genres_prediction(k_nn, dataset):
  sum = 0
  for x in k_nn:
    genres = dataset[dataset.movieid == x]['genre'].tolist()[0]
    sum = sum + len(genres)
  return round(1.2 * sum / len(k_nn))


def sort_index(lst, rev=True):
  index = range(len(lst))
  s = sorted(index, reverse=rev, key=lambda i: lst[i])
  return s


def get_index_of_top_labels(out, number_of_genres):
  out_index = sort_index(out)
  output = out_index[:number_of_genres]
  return output


def genres_prediction_from_rating(graph, movieid, list_classes, n, dataset):
  out = [0 for i in range(len(list_classes))]

  if movieid not in list(graph.nodes):
    return out

  if len(graph.edges(movieid)) <= 0:
    return out

  k_nn = knn(graph, movieid, n, movies_test['movieid'].tolist())

  if len(k_nn) == 0:
    return out

  j = 0
  for genre in list_classes:
    sum = 0
    for i in range(len(k_nn)):
      w = G.get_edge_data(movieid, k_nn[i])['weight']
      # s = 2 *w[0] * w[1] / (w[0] + w[1])
      s = w[0] * w[1]
      genre_of_neighbor = dataset[dataset.movieid == k_nn[i]]['genre'].tolist()[0]
      gamma = 0
      if genre in genre_of_neighbor:
        gamma = 1
      sum = sum + gamma * s
    sum = sum / len(k_nn)
    out[j] = sum
    j = j + 1

  number_of_genres = number_of_genres_prediction(k_nn, dataset)

  top_index = get_index_of_top_labels(out, number_of_genres)

  out = [0. for i in range(len(list_classes))]

  for i in top_index:
    out[i] = 1.

  return out

### Load genres prediction based on user-ratings (You need to run this cell)

We have constructed the result of genres prediction based on user-ratings for saving time when training. You only need to run this cell.

In [ ]:
!gdown 1oq4MuyGSyYpvhyAFLBcpCAhi04bGQctT

Downloading...
From: https://drive.google.com/uc?id=1oq4MuyGSyYpvhyAFLBcpCAhi04bGQctT
To: /content/rating_result
100% 615k/615k [00:00<00:00, 6.07MB/s]


In [ ]:
with open('/content/rating_result', 'rb') as f:
  rating_result = pickle.load(f)

## Proposed model

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, num_classes, len_vocab, embedding_dimension=3898, hidden_size=64):
        super(BaseModel, self).__init__()
        self.hidden_size = hidden_size

        self.resnet101 = models.resnet101(pretrained=True)
        in_features = self.resnet101.fc.in_features

        self.resnet101.fc = nn.Sequential(
            nn.Linear(in_features, hidden_size * 2),
            nn.PReLU(),
            nn.BatchNorm1d(hidden_size * 2),
            nn.Linear(hidden_size * 2, hidden_size)
        )

        self.lstm = nn.LSTM(3899, hidden_size, 4, batch_first=True, bidirectional=False)

        self.fc1 = nn.Linear(hidden_size, 64)

        self.fc2 = nn.Sequential(
            nn.Linear(2*hidden_size + num_classes, 2*hidden_size),
            nn.PReLU(),
            nn.BatchNorm1d(2*hidden_size, affine = False),
            nn.Linear(hidden_size*2, hidden_size),
            nn.PReLU(),
            nn.BatchNorm1d(hidden_size, affine = False),
            nn.Linear(hidden_size, 32),
            nn.PReLU(),
            nn.BatchNorm1d(32, affine = False),
            nn.Linear(32, num_classes),
            nn.Sigmoid()
        )

    def forward(self, text_tens, img_tensor, rating_feat, attention_mask=None):
        img_feat = self.resnet101(img_tensor)

        lstm_out, _ = self.lstm(text_tens)

        last_output = lstm_out[:, -1, :]

        text_feat = self.fc1(last_output)

        combined_feat = torch.cat([img_feat, text_feat, rating_feat], 1)

        out = self.fc2(combined_feat)

        return out

In [ ]:
with open('ml1m/content/dataset/genres.txt', 'r') as f:
    genre_all = f.readlines()
num_classes = len(genre_all)
num_classes

vocab = create_vocab()
model = BaseModel(num_classes, len(vocab))

## Loss function and experiment setting

In [ ]:
learning_rate = 1e-4
optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=learning_rate,
    weight_decay=1e-5
)

## Training

In [ ]:
device = torch.device('cuda:0')
NUM_EP = 20
model.to(device)

criterion = nn.BCELoss()

for ep in range(NUM_EP):
    print('ep:', ep)

    for idx, (title_tensor, img_tensor, genre_tensor, index) in enumerate(train_dataloader):
        title_tensor = title_tensor.to(device)
        genre_tensor = genre_tensor.to(device)
        img_tensor = img_tensor.to(device)

        rating_feat = []
        for i in index:
          rating_feat.append(rating_result[i.item()])

        rating_feat = torch.tensor(rating_feat)
        rating_feat = rating_feat.to(device)

        out = model(title_tensor, img_tensor, rating_feat)

        optimizer.zero_grad()
        loss = criterion(out, genre_tensor)

        loss.backward()
        optimizer.step()

    #print(loss)

ep: 0
ep: 1
ep: 2
ep: 3
ep: 4
ep: 5
ep: 6
ep: 7
ep: 8
ep: 9
ep: 10
ep: 11
ep: 12
ep: 13
ep: 14
ep: 15
ep: 16
ep: 17
ep: 18
ep: 19


## Evaluation

The proposed model

In [ ]:
N, C = genre_tensor.shape

auroc_all = 0
f1_all = 0
pre_all = 0
rec_all = 0
acc_all = 0
map = 0
f1 = F1Score(task='multilabel', num_labels=C, threshold=0.62, average='macro')
pre = Precision(task='multilabel', num_labels=C, threshold=0.62, average='macro')
rec = Recall(task='multilabel', num_labels=C, threshold=0.62, average='macro')
acc = Accuracy(task='multilabel', num_labels=C)
f1 = f1.to(device)
pre.to(device)
rec.to(device)
acc.to(device)

for title_tensor, img_tensor, genre_tensor, index in test_dataloader:
      title_tensor = title_tensor.to(device)
      img_tensor = img_tensor.to(device)
      genre_tensor = genre_tensor.to(device)

      rating_feat = []
      for i in index:
        rating_feat.append(rating_result[i.item()])

      rating_feat = torch.tensor(rating_feat)
      rating_feat = rating_feat.to(device)
      #out = rating_feat
      out = model(title_tensor, img_tensor, rating_feat)

      f1_batch = f1(out, genre_tensor)
      pre_batch = pre(out, genre_tensor)
      rec_batch = rec(out, genre_tensor)
      acc_batch = acc(out, genre_tensor)
      f1_all += f1_batch
      pre_all += pre_batch
      rec_all += rec_batch
      acc_all += acc_batch
      map += retrieval_average_precision(out, genre_tensor)

print('F1 score: ', f1_all/len(test_dataloader))
print('Precision: ', pre_all/len(test_dataloader))
print('Recall: ', rec_all/len(test_dataloader))
print('Accuracy: ', acc_all/len(test_dataloader))
print('MAP@k: ', map/len(test_dataloader))

F1 score:  tensor(0.3072, device='cuda:0')
Precision:  tensor(0.2886, device='cuda:0')
Recall:  tensor(0.3734, device='cuda:0')
Accuracy:  tensor(0.8244, device='cuda:0')
MAP@k:  tensor(0.5982, device='cuda:0')


Movie genres prediction result based on user-ratings between $k=20$ and $k=40$

In [ ]:
N, C = genre_tensor.shape

auroc_all = 0
f1_all = 0
pre_all = 0
rec_all = 0
acc_all = 0
accuracy = 0
f1 = F1Score(task='multilabel', num_labels=C, threshold=0.62, average='weighted')
pre = Precision(task='multilabel', num_labels=C, threshold=0.62, average='weighted')
rec = Recall(task='multilabel', num_labels=C, threshold=0.62, average='weighted')
acc = Accuracy(task='multilabel', num_labels=C)
map = 0
f1 = f1.to(device)
pre.to(device)
rec.to(device)
acc.to(device)

for title_tensor, img_tensor, genre_tensor, index in test_dataloader:
      title_tensor = title_tensor.to(device)
      img_tensor = img_tensor.to(device)
      genre_tensor = genre_tensor.to(device)

      rating_feat_20 = []
      rating_feat_40 = []
      for i in index:
        rating_feat_20.append(rating_result[i.item()])
        #rating_feat_40.append(genres_prediction_from_rating(G, i.item(), genre_all, 40, dataset))

      rating_feat_20 = torch.tensor(rating_feat_20)
      rating_feat_20 = rating_feat_20.to(device)
      rating_feat_40 = torch.tensor(rating_feat_40)
      rating_feat_40 = rating_feat_40.to(device)
      out = rating_feat_20

      f1_batch = f1(out, genre_tensor)
      pre_batch = pre(out, genre_tensor)
      rec_batch = rec(out, genre_tensor)
      acc_batch = acc(out, genre_tensor)
      f1_all += f1_batch
      pre_all += pre_batch
      rec_all += rec_batch
      acc_all += acc_batch
      map += retrieval_average_precision(out, genre_tensor)

print('F1 score: ', f1_all/len(test_dataloader))
print('Precision: ', pre_all/len(test_dataloader))
print('Recall: ', rec_all/len(test_dataloader))
print('Accuracy: ', acc_all/len(test_dataloader))
print('MAP@k: ', map/len(test_dataloader))


F1 score:  tensor(0.6549, device='cuda:0')
Precision:  tensor(0.6053, device='cuda:0')
Recall:  tensor(0.7715, device='cuda:0')
Accuracy:  tensor(0.9233, device='cuda:0')
MAP@k:  tensor(0.4910, device='cuda:0')
